In [ ]:
import numpy as np
from safeable import *
from advisoryParams import *
from plotPolicy import *
from matplotlib2tikz import save as tikz_save # For saving to tikz for LaTex

%load_ext autoreload
%autoreload 2
%matplotlib inline

### Running Example

Feel free to change values and re-run everything

In [ ]:
pra = SCL1500
ra = CL1500
pd = 0
eps= 1
v = [-15,-5]

advisory = advisoryParams(ra,pd=pd,eps=eps)

### Plot safeable boundaries

In [ ]:
trajMin,trajMax,boundMin,boundMax = getSafeable(advisory,v,worstCase=False)
ax = plotBounds(boundMin,boundMax,trajMin,trajMax,withTraj=True,c='k',name="Safeable Example")

### Assume worst case tau

In [ ]:
trajMin,trajMax,boundMin,boundMax = getSafeable(advisory,v,worstCase=True)  # worstCase is true by default
ax = plotBounds(boundMin,boundMax,trajMin,trajMax,withTraj=True, c='r', name="Safeable Example, Worst Case")

### Combine plots onto one axes

In [ ]:
# This shows that you can turn off the trajectories, add labels, and draw the legend
trajMin,trajMax,boundMin,boundMax = getSafeable(advisory,v,worstCase=False)
ax = plotBounds(boundMin,boundMax,trajMin,trajMax,withTraj=False,c='k',name="Safeable Example", label="Original Safeable")

trajMin,trajMax,boundMin,boundMax = getSafeable(advisory,v,worstCase=True)  # worstCase is true by default
plotBounds(boundMin,boundMax,trajMin,trajMax,axes = ax, withTraj=False, c='r--', name="Safeable Example", label="Worst Case")
ax.legend()

### Plot all Safeable regions on one axes

In [ ]:
plotUnsafe = True # Set to true if you also want to show the unsafe-for-all region
plotAll(pra,v,plotUnsafe=plotUnsafe)

### Plot safeable advisory and the unsafe-for-all region

In [ ]:
_,_,boundMin,boundMax = getSafeable(advisory,v,worstCase=True)  # worstCase is true by default
axes=plotBounds(boundMin,boundMax,name="Safeable Example", label="Safeable")

boundMin,boundMax = getUnsafeRegion(pra,v,pd,eps)
axes = plotBounds(boundMin,boundMax,axes=axes,c='r',label='Unsafe for all')
axes.legend()

### Plot unsafeable region

In [ ]:
plotGetRegionToCheck(pra,ra,v,pd,eps)

### Plot safeable regions with colors filled

In [ ]:
nbin = 350
plotSafeableAdvisory(pra,ra,v,pd,eps,nbin=nbin,withTraj=True)

### Plot under/over approximations of the unsafeable regions

Linearization resolution

* dt: Full length of line segment

* dti: Spacing of linear bounds

dt should always be an integer multiple of dti and **at least twice** dti

In [ ]:
dt = 1.25
dti=0.0625

In [ ]:
ax = plotGetRegionToCheck(pra,ra,v,pd,eps,label="True")
plotGetRegionToCheck(pra,ra,v,pd,eps,c='c',axes=ax,name="Over Approximate",label="Approx",approx=True,overApprox=True,dt=dt,dti=dti)
ax.legend()

In [ ]:
ax = plotGetRegionToCheck(pra,ra,v,pd,eps,label="True")
plotGetRegionToCheck(pra,ra,v,pd,eps,label="Approx",c='y',axes=ax,name="Under Approximate",approx=True,overApprox=False,dt=1.0,dti=0.0625)
ax.legend()

### Plot policy with unsafeable region

In [ ]:
plotSafeablePolicy(pra,ra,v,pd,eps,useTable=False)

### How to save to tikz for use in LaTex

Adding the tikz_save command after any of these plots will save the plot to a tex plot.

In [ ]:
plotSafeablePolicy(pra,ra,v,pd,eps,useTable=False,drawLegend=False,)
tikz_save("Pics/SafeableExample.tex")